In [8]:
path = join_path((pathlst['measure'],[basename,'rok']))
data_rok = load_safe(path)

def poissonfit(arr):
    from scipy.optimize import curve_fit
    from scipy.stats import poisson
    y, bin_edges = np.histogram(arr, bins=freq_bins - 0.5, density=True)
    bin_middles = freq_bins.astype(float)[:-1]#(bin_edges[1:] + bin_edges[:-1])/2
    fit_function = lambda k, lamb:poisson.pmf(k, lamb)
    parameters, cov_matrix = curve_fit(fit_function, bin_middles, y)
    yfit = fit_function(freq_bins, *parameters)
    return y,yfit


path = join_path((pathlst['measure'],[basename,'rok']))
data_rok = load_safe(path)

speed_bins = np.arange(20)
freq_bins = np.arange(12)
recur_bins = np.arange(51)

stacked = [get(data_rok,'speed',np.hstack,lambda phenotype,x: np.histogram(x,bins=speed_bins,density=True)[0]),
           get(data_rok,'speed',None,lambda phenotype,x: 1-(np.histogram(x,bins=speed_bins,density=True)[0][:3]).sum()),
           get(data_rok,'duration','arr',None),
           get(data_rok,'area','arr',None),
           get(data_rok,'freq',np.hstack,lambda phenotype,x: poissonfit(x)),
           get(data_rok,'recur',np.hstack,lambda phenotype,x: np.histogram(x,bins=recur_bins,density=True)[0])
          ]


dict_keys(['20190524_3', '20191108_18', '20191028_15', '20190803_19', '20191108_15', '20190524_7', '20191026_10', '20191027_7', '20191026_8', '20191129_12', '20190524_4', '20191129_16', '20191129_9', '20190803_17', '20191108_19', '20191129_5', '20191028_18', '20190524_5', '20191129_14', '20190803_18', '20191129_6', '20190729_12', '20190729_10', '20190803_13', '20191028_17', '20190729_11', '20191026_9', '20190803_11', '20191026_6', '20190803_9', '20191108_22', '20190729_13', '20190803_14', '20190803_7', '20190729_15', '20190524_8', '20191026_7', '20191108_16', '20191129_15', '20191028_19', '20190803_8', '20191028_8', '20190524_6', '20191027_8']) dict_keys(['20190524_3', '20191108_18', '20191028_15', '20190803_19', '20191108_15', '20190524_7', '20191026_10', '20191027_7', '20191026_8', '20191129_12', '20190524_4', '20191129_16', '20191129_9', '20190803_17', '20191108_19', '20191129_5', '20191028_18', '20190524_5', '20191129_14', '20190803_18', '20191129_6', '20190729_12', '20190729_10', 

In [28]:

def rok_ts_kmgh(nzzs,img_raw,RC):
    from imagestack import blur_stk
    img_raw = blur_stk(img_raw,4,divide(1.2,tRes))

    from imagestack import crop_group_z
    zs,zgroups = crop_group_z(nzzs,tRes)

    from rotate_crop import rotate_crop,kmgh
    img = rotate_crop(img_raw,zs,RC)

    img_kmgh, x_kmgh = kmgh(img,tRes,H=7)

    return img,img_kmgh,x_kmgh

def figure1():# SD embryo ROK raw and kymograph
    minmax_g_dict = {'spd':(100,900),'nmy':(400,900),'mel11':(200,700),'plst':(200,800),
                    'unc60':(200,900),'cyk':(300,750),'pfn':(300,800),'ani':(300,900)}

    IDs = infolst[infolst.apply(lambda x: 
        str(x['comment']).startswith('EXAMPLE_')
        , axis=1)].index.values.astype(str)

    print(infolst.loc[IDs])
    movies = {}
    for ID in IDs:
        globals().update({'ID':ID, **dict(infolst.loc[ID])})

        result_raw = load_safe(join_path((pathlst['pipeline'],'run_raw',[phenotype,ID])))
        img,img_kmgh,x_kmgh = rok_ts_kmgh(result_raw['nzzs'][2:],result_raw['img'],result_raw['RC'])

        minmax_g = minmax_g_dict[phenotype]
        from colorcode import standarize, standarized_to_rgb
        img,img_kmgh = standarized_to_rgb(standarize(img, *minmax_g)),standarized_to_rgb(standarize(img_kmgh, *minmax_g))

        idx_kmgh = int(comment.split('_')[1])-1
        tifffile.imsave(safe_path(pjoin(pathlst['paper'],'f1','-'.join([phenotype,ID])+'_img_kmgh.tif')), img_kmgh[idx_kmgh])
        
        imgg = img.copy()
        #imgg[:,x_kmgh[idx_kmgh]] = 255
        tifffile.imsave(join_path((pathlst['paper'],'f1','-'.join([phenotype,ID])+'_img.tif')), imgg)

        movies[phenotype] = img
    return movies

def figure1_stack_save(movies, info):
    H2 = 40
    titleshape = list(movies['spd'].shape)
    titleshape[1] = H2
    title = np.full(titleshape,0,dtype=np.uint8)

    row1 = [np.hstack([title,np.hstack([title,movies[phenotype]])]) for phenotype in info.keys()]
    color = (255,255,255)
    import cv2
    dt = 0.6
    for i in range(row1[0].shape[0]):
        cv2.putText(row1[0][i], str(round(dt*i,1))+' S', (30,H2-10), cv2.FONT_HERSHEY_SIMPLEX , 1, color, 2)
        cv2.line(row1[0][i], (20,240+H2), (20+50,240+H2), color, 2)# scale bar 5um
        for j,txt in enumerate(info.values()):
            cv2.putText(row1[j][i], txt, (120,2*H2-10), cv2.FONT_HERSHEY_SIMPLEX , 1, color, 2)
    from colorcode import stk_boarder,stk_upsizexy
    im = stk_upsizexy(np.dstack([stk_boarder(temp) for temp in row1]))
    
    saveMP4(join_path((pathlst['paper'],'movies',['raw','_'.join(info.keys())])),im,40)

#'''
movies=figure1()
figure1_stack_save(movies, {'spd':'Control','cyk':'cyk-1(RNAi)','pfn':'pfn-1(RNAi)','ani':'ani-1(RNAi)'})
figure1_stack_save(movies, {'spd':'Control','unc60':'unc-60(RNAi)'})
figure1_stack_save(movies, {'spd':'Control','nmy':'nmy-2(ts)','mel11':'mel-11(RNAi)','plst':'plst-1(tm2455)'})
#'''


            phenotype LabelG LabelR  tRes  nEnd    note contractility  \
ID                                                                      
20210409_7       plst    ROK    NMY   0.6   400  SD3050           NaN   
20210410_11     unc60    ROK    NMY   0.6   300  SD3050           NaN   
20210420_29       spd    ROK    NMY   0.6   350  SD3050           NaN   
20210420_23     mel11    ROK    NMY   0.6   250  SD3050           NaN   
20190729_10       ani    ROK     LA   0.6   450  SD3050           NaN   
20190803_19       cyk    ROK     LA   0.6   350  SD3050  intermediate   
20191026_8        pfn    ROK     LA   0.6   350  SD3050       strong2   
20191108_15       nmy    ROK   None   0.6   520    SD30           NaN   

                comment  
ID                       
20210409_7    EXAMPLE_6  
20210410_11  EXAMPLE_10  
20210420_29  EXAMPLE_11  
20210420_23   EXAMPLE_7  
20190729_10   EXAMPLE_5  
20190803_19  EXAMPLE_13  
20191026_8    EXAMPLE_9  
20191108_15   EXAMPLE_9  
dict_keys

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (5792, 1288) to (5792, 1296) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2896, 1288) to (2896, 1296) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (5792, 1288) to (5792, 1296) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [22]:
def test_significance(data):
    from scipy.stats import mannwhitneyu
    for phenotype1,phenotype2 in [
        ('spd','cyk'),('spd','ani'),('cyk','ani'),
        ('spd','nmy'),('cyk','cyk+nmy'),('ani','ani+nmy')]:
        data1,data2 = data[phenotype1],data[phenotype2]
        stat, p = mannwhitneyu(data1,data2)
        print(phenotype1,phenotype2,round(p,3),len(data1),len(data2))
    
    
for name in ['speed','duration','area','freq','recur']:
    print(name)
    test_significance(get(data_rok,name,np.hstack,None))
    print()


speed
spd cyk 0.0 45496 82638
spd ani 0.0 45496 47574
cyk ani 0.0 82638 47574
spd nmy 0.0 45496 54105
cyk cyk+nmy 0.0 82638 62923
ani ani+nmy 0.0 47574 86216

duration
spd cyk 0.06 6 8
spd ani 0.009 6 5
cyk ani 0.413 8 5
spd nmy 0.009 6 7
cyk cyk+nmy 0.062 8 5
ani ani+nmy 0.126 5 7

area
spd cyk 0.001 6 8
spd ani 0.004 6 5
cyk ani 0.357 8 5
spd nmy 0.309 6 7
cyk cyk+nmy 0.002 8 5
ani ani+nmy 0.165 5 7

freq
spd cyk 0.0 615914 561054
spd ani 0.0 615914 349737
cyk ani 0.0 561054 349737
spd nmy 0.0 615914 607577
cyk cyk+nmy 0.0 561054 562565
ani ani+nmy 0.0 349737 561507

recur
spd cyk 0.0 555293 1816302
spd ani 0.0 555293 941121
cyk ani 0.0 1816302 941121
spd nmy 0.0 555293 742353
cyk cyk+nmy 0.0 1816302 1000105
ani ani+nmy 0.0 941121 1755149



In [ ]:


def fade(msk,N):
    result = np.full(msk.shape,np.nan,dtype=np.float32)
    for z in range(N,msk.shape[0]):
        zs = np.arange(z-N,z+1)
        for zz in zs:
            result[z,msk[zz]] = (zz-zs[0])/(zs[-1]-zs[0])
    return result

def fade_msks(rgb,msks,outline,RC):
    N = 3
    for msk,color in msks:
        imshape = msk.shape
        color = matplotlib.colors.to_rgb(color)
        faded = fade(msk,N)
        m = ~np.isnan(faded)
        rgb[m] = gradient_saturated_gray(color,235/255,faded,-0.5,1,reverse=True)[m]
    
    rgb[:,~outline] = 255
    from rotate_crop import rgb_rotate_crop
    rgb = rgb_rotate_crop(rgb,None,RC)
    return rgb

def draw_line(im,xy0,xy1,value,thickness):
    mask = np.zeros(im.shape[:2],dtype=np.uint8)
    import cv2
    mask = cv2.line(mask,tuple(np.flip(xy0)),tuple(np.flip(xy1)),255,thickness)>0
    im[np.where(mask)]=value
    return im

def cc_cum(graph,cc,blobs,imshape,outline,RC):
    for pz in blobs.keys():
        for b in blobs[pz]:
            b.center = b.pixels.mean(0).astype(int)
    darkcolors = dark_unique_colors()
    result = np.full(list(imshape[1:])+[3],235,dtype=np.uint8)
    from spread import component_duration,component_get_zxy
    cc = [component for component in cc if component_duration(component)>5]
    print(len(cc))
    import cv2
    for rank,component in enumerate(cc):
        color = gradient_saturated_gray(darkcolors[rank],1,0.2,0,1,reverse=False)
        color = ( int (color [ 0 ]), int (color [ 1 ]), int (color [ 2 ])) 
        
        for pz,j in component:
            b = blobs[pz][j]
            b_center = b.center
            for npz,nj in graph.neighbors((pz,j)):
                n_center = blobs[npz][nj].center
                cv2.line(result,tuple(np.flip(b_center)),tuple(np.flip(n_center)),color,2)
    result[~outline] = 255
    RC.theta = 0
    from rotate_crop import rgb_rotate_crop
    result = rgb_rotate_crop(result,None,RC)
    return result

def cc_cum_largest(graph,cc,blobs,imshape,outline,RC):
    darkcolors = dark_unique_colors()
    result = np.full(list(imshape[1:])+[3],235,dtype=np.uint8)
    rank = 0
    color = gradient_saturated_gray(darkcolors[rank],1,0.2,0,1,reverse=False)
    from spread import component_get_zxy
    zxy = component_get_zxy(blobs,cc[0])
    result[zxy[:,1],zxy[:,2]] = color
    result[~outline] = 255
    RC.theta = 0
    from rotate_crop import rgb_rotate_crop
    result = rgb_rotate_crop(result,None,RC)
    return result

def cc_cum_kmgh(graph,cc,blobs,imshape,outline,RC):
    darkcolors = dark_unique_colors()
    result = np.full(list(imshape)+[3],235,dtype=np.uint8)
    from spread import component_duration,component_get_zxy
    cc = [component for component in cc if component_duration(component)>5]
    for rank,component in enumerate(cc):
        color = gradient_saturated_gray(darkcolors[rank],1,0.2,0,1,reverse=False)
        zxy = component_get_zxy(blobs,component)
        result[zxy[:,0],zxy[:,1],zxy[:,2]] = color
    result[:,~outline] = 255
    RC.theta = 0
    from rotate_crop import rgb_rotate_crop
    result = rgb_rotate_crop(result,None,RC)
    return result[:,120]


IDs = ['20190524_7','20190803_19','20190729_12']#

for ID in IDs:
    data = data_examples[ID]
    phenotype = infolst.loc[ID]['phenotype']
    im_cc_cum = cc_cum(data['graph'],data['cc'],data['blobs'],data['imshape'],data['outline'],data['RC'])
    path = safe_path(pjoin(pathlst['paper'],'f3','cccum_'+phenotype+'.tif'))
    tifffile.imsave(path,im_cc_cum)

    im_cc_cum_largest = cc_cum_largest(data['graph'],data['cc'],data['blobs'],data['imshape'],data['outline'],data['RC'])
    path = safe_path(pjoin(pathlst['paper'],'f3','cccumlargest_'+phenotype+'.tif'))
    tifffile.imsave(path,im_cc_cum_largest)
    
    im_cc_cum_kmgh = cc_cum_kmgh(data['graph'],data['cc'],data['blobs'],data['imshape'],data['outline'],data['RC'])
    path = safe_path(pjoin(pathlst['paper'],'f3','cccumkmgh_'+phenotype+'.tif'))
    tifffile.imsave(path,im_cc_cum_kmgh)


    from velocity import erodemask
    erode_outline = erodemask(data['outline'],10)

    actin = data['R']['raw_disk'][:100]
    actin = standarize_colormap(actin,40,800,'gray')
    
    msk_slow,msk_fast = msk_speed(data['points'],data['imshape'])
    msk_slow[:,~erode_outline] = False
    msk_fast[:,~erode_outline] = False
    tifffile.imsave('speed_'+infolst.loc[ID]['phenotype']+'.tif',
                    fade_msks(actin.copy(),((msk_fast,'darkorange'),(msk_slow,'dodgerblue')),outline,data['RC']))    
    
    msk_short,msk_long = msk_recurrence(data['NEWEXC'],outline)
    msk_short[:,~erode_outline] = False
    msk_long[:,~erode_outline] = False
    tifffile.imsave('recur_'+infolst.loc[ID]['phenotype']+'.tif',
                    fade_msks(actin.copy(),((msk_short,'darkorange'),(msk_long,'dodgerblue')),outline,data['RC']))
    
def fig_pfn(fig,recur):
    N = len(recur['pfn'])
    axes3 = addax(fig,L=1,W=1,T=1,H=N,ncols=1,nrows=N,wspace=2,hspace=0.2,sharex='col',sharey='col')#,sharey='all'
    axes3tw = addax(fig,L=2.5,W=0.2,T=1,H=N,ncols=1,nrows=N,wspace=2,hspace=0.15,sharex='col',sharey='col')
    
    recur_bins = np.arange(51)
    
    from histogram import step
    for i,(ID,arr) in enumerate(recur['pfn'].items()):
        recur_hist = np.histogram(arr,bins=recur_bins,density=True)[0]
        x,y = step(recur_bins*1.2,recur_hist).T
        axes3[i].fill_between(x[:-2],y[:-2],color='k',alpha=0.5,lw=0)
        axes3tw[i].plot(x[-1],y[-1],'x',color='k')
        
    axes3[-1].set(xlim=[0,60],xlabel='      Time before re-activation (s)',ylim=[0,0.06],xticks=[0,20,40,60])
    axes3[int(N/2)].set(ylabel='Density')
    for ax3tw in axes3tw:
        ax3tw.set(xlim=[59,61],xticks=[60],xticklabels=['>60'],ylim=[0,0.6],yticks=[0,0.5])
        
        

figure(join_path((pathlst['paper'],'f3')),[get(data_rok,'recur',None,None)],fig_pfn)
